# Spark ML Production

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("Spark ML Production")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/27 19:29:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/27 19:29:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Загружаем данные

In [2]:
data = spark\
    .read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("data/BankChurners.csv")

## Загружаем модель

In [3]:
from pyspark.ml import PipelineModel

model = PipelineModel.load("pipelineModel")

## Вычисляем

In [4]:
predicted = model.transform(data)

In [5]:
predicted.show(5)

23/11/27 19:29:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

In [8]:
predicted.select('prediction', 'Attrition_Flag').show()

+----------+-----------------+
|prediction|   Attrition_Flag|
+----------+-----------------+
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       1.0|Existing Customer|
|       0.0|Existing Customer|
|       1.0|Existing Customer|
|       0.0|Existing Customer|
|       1.0|Existing Customer|
|       1.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       1.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
|       0.0|Existing Customer|
+----------+-----------------+
only showing top 20 rows



In [9]:
predicted.select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 7174|
|       1.0| 2953|
+----------+-----+



## Проверяем результат

In [10]:
tp = predicted.filter((f.col("Attrition_Flag") == "Attrited Customer") & (f.col("prediction") == 1)).count()
tn = predicted.filter((f.col("Attrition_Flag") == "Existing Customer") & (f.col("prediction") == 0)).count()
fp = predicted.filter((f.col("Attrition_Flag") == "Existing Customer") & (f.col("prediction") == 1)).count()
fn = predicted.filter((f.col("Attrition_Flag") == "Attrited Customer") & (f.col("prediction") == 0)).count()

print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")

Confusion Matrix:
1325	1628
 302	6872


In [11]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"Accuracy = {accuracy:1.4f}")
print(f"Precision = {precision:1.4f}")
print(f"Recall = {recall:1.4f}")

Accuracy = 0.8094
Precision = 0.4487
Recall = 0.8144


23/11/28 01:05:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 907500 ms exceeds timeout 120000 ms
23/11/28 01:05:43 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/28 01:05:45 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B